# **Sistema Lineares**
---
<ul>
  <li><a href="#scrollTo=6T7Gy-Docn5C&uniqifier=1">Sistemas Lineares</a>
  </li>
  <ul>
    <li><a href="#scrollTo=RoOsu9Ppsd0v&uniqifier=1">Funções comuns</a></li>
    <li><a href="#scrollTo=6X8hhhuyhZHC&uniqifier=1">Método direto</a>
      <ul>
          <li><a href="#scrollTo=fvXmmrPMwIVa&uniqifier=1">Métodos</a></li>
          <li><a href="#scrollTo=6X8hhhuyhZHC&uniqifier=1">Comparações</a></li>
      </ul>
    </li>
    <li><a href="#scrollTo=Ti2W5pc9VLcl&uniqifier=1">Método iterativo</a>
        <ul>
          <li><a href="#scrollTo=6X8hhhuyhZHC&uniqifier=1">Métodos</a></li>
          <li><a href="#scrollTo=6X8hhhuyhZHC&uniqifier=1">Comparações</a></li>
      </ul>
    </li>
    <li><a href="#scrollTo=6X8hhhuyhZHC&uniqifier=1">Aplicações</a></li>
  </ul>

In [ ]:
import time

def timeit(method):
    """
      @Decorator
      Retorna o tempo de excução da função
      Não modifica a função.
    """
    def timed(*args, **kw):
        ts = time.time();
        result = method(*args, **kw);
        te = time.time();
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__);
            kw['log_time'][name] = (te - ts) * 1000;
        else:
            print("%r  %2.5f ms" % (method.__name__, (te - ts) * 1000));
        return result;
    return timed;

# Introdução

> $Ax = b$

"O vetor $b_m$ pode ser expresso como combinação linear das colunas de $A_{mn}$?"[1] 

> $a_{11}*x_{1} + \cdots + a_{1n}*x_{n} = b_{1}$

> $a_{21}*x_{1} + \cdots + a_{2n}*x_{n}= b_{2}$

> $\vdots$ 

> $a_{m1}*x_{1} + \cdots + a_{mn}*x_{n} = b_{m}$

Para facilitar, usaremos majoritariamente matrizes não-singulares, $|A| != 0$ de rank(A) = n, e portanto, haverá sempre ao menos uma solução.

> $x = -A^{-1}*b$

# Funções Comuns


**Foward Solution Ly = b**

>$L_{m,n} =
 \begin{pmatrix}
  l_{1,1} & 0 & \cdots & 0 \\
  l_{2,1} & l_{2,2} & \cdots & 0 \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  l_{m,1} & l_{m,2} & \cdots & l_{m,n}
 \end{pmatrix}$


> $y_1 = b_1/l_{11}$

>$\vdots$

> $y_{n} = b_n - \sum_{k=1}^{n} l_{ik}*y_{ik}$ 

In [ ]:
# Ly = b, return y of size n
def forward_substitution(A, b):
  l = np.copy(A);
  y = np.copy(b); 
  n = np.shape(A)[0];

  for i in range(1, n):
    y[i] -= np.dot(l[i, 0:i], y[0:i]);
  return y;

**Back Solution Ux = y (then)**

>$U_{m,n} =
 \begin{pmatrix}
  u_{1,1} & u_{1,2} & \cdots & u_{1,n} \\
  0 & a_{2,2} & \cdots & u_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  0 & 0 & \cdots & u_{m,n}
 \end{pmatrix}$


> $x_n = y_n/u_{nn}$

> $x_{n-1} = (y_{n-1}-x_{n}*u_{n-1n})/u_{n-1n-1}$ 


In [ ]:
# Ux = y, return x is array size n
def back_substituion(A, y): 
  u = np.copy(A);
  x = np.copy(y);
  n = len(x);

  for i in range(n-1, -1, -1):
    x[i] = (x[i] - np.dot(u[i,i+1:n], x[i+1:n]))/u[i, i];

  return x;

In [ ]:
# A is a quadratic matriz, return the transversed A
def transverse(A):
  n = len(A)
  for i in range(0, n):
    b = np.copy(A[i, i:n])
    A[i, i:n] = np.copy(A[i:n, i])
    A[i:n, i] = np.copy(b)
  return A

In [ ]:
# U: matriz triangular superior quadrada 
def determinante_upper_matrix(U):
  det = 1
  n = np.shape(U)[0];

  for i in arange(0, n-1):
    det *= U[i, i];
  return det;

# **Métodos diretos**


**Solve Ax = b**

> $Ax = (LU)x = b$

> $L(Ux) = b$

> $Ly = b$    // foward solution

> $Ux = y$    // back solution

In [ ]:
# Recebe a matriz A, o vetor b e a função de decomposição A = (LU)
def lu_solve_system(A, b, decomposition):
    L, U = decomposition(A);
    y = forward_substitution(L, b);
    x = back_substituion(U, y);
    return x

## O método da decomposição LU

Decompoẽ a matriz A em LU.

>$A_{m,n} =
 \begin{pmatrix}
  u_{1,1} & u_{1,2} & u_{1,3} \\
  u_{1,1}*l_{2,1} & u_{1,2}*l_{2,1} + u_{2,2}  & u_{1,3}*l_{2,1} + u_{2,3} \\
  u_{1,1}*l_{3,1} & u_{12}*l_{3,1} + u_{2,2}*l_{3,2} &  u_{13}*l_{3,3} + u_{2,3}*l_{3,2} + u_{3,3}
 \end{pmatrix}$

In [ ]:
# Recebe a matriz A, retorna a matriz aumentada [L/U]
def LU_decomposition(A):
  LU = np.copy(A);
  n = len(A);

  for k in range(0, n-1):
    for i in range(k+1, n):
      if(A[i, k]!=0):
        m = A[i, k]/A[k, k]
        A[i, k+1:n] -= m*A[k, k+1:n];
        A[i, k] = m;

  return A

## O método de Gaus


O método de Gaus consiste em obter a matriz triangular inferior utilizando apenas operações elementares na forma matricial.

Elas são, sobre a matriz A:

1. Tocar duas linhas de A, o que muda o sinal de |A|.
2. Multiplicar uma linha por um escalar $a$, diferente de zero, o que implica $a|A|$.
3. Multiplicar uma linha e subtrair de uma linha. 

Formula geral:
> $A_{ij} = A_{ij} +- a * A_{kj}, j = k, k + 1, ..., n$

> $b_i = b_i - a*b_k$

Com A transformada em U, agora podemos fazer:
> $Ux = y$

## O método de Gaus com pivoteamento

In [ ]:
# Recebe a matriz A, os indexes i e j, referente as duas linhas que comutarão.
def swap_row(A, i, j):
  if(len(A.shape) == 1):
    A[i], A[j] = A[j], A[i]
  else:
    tmp = np.copy(A[i, :])
    A[i, :] = np.copy(A[j, :])
    A[j, :] = np.copy(tmp[:])

In [ ]:
# Recebe a matriz A, e o vetor b. Returna [U|y] (matriz aumentada)
def gaussPivot(A, b, tol=1.0e-9):
  U = np.copy(A);
  y = np.copy(b);
  n = len(b);

  s = np.zeros([n])
  for i in range(0, n):
    s[i] = np.max(np.abs(A[i, :]))
  
  #pivoteamento
  for k in np.arange(n-1):
    p = np.argmax(np.abs(A[k, k:n]/s[k:n]))
    if(np.abs(A[p, k]) < tol):
      error.err('Matriz singular!')
    if(p != k):
      swap_row(A, p, k)
      swap_row(b, p, k)
      swap_row(s, p, k)

  #elimination
    for i in range(k+1, n):
      m = 0.0;
      if(A[i, k] != 0):
        m = A[i, k]/A[k, k]
        A[i, k+1:n] -= m*A[k, k+1:n];
        b[i] -= m*b[k]; 

  return [A, b]

## O método de Cholesks

Esse método decompoẽ uma matriz simétrica e positiva definida.

> $A = LL^t$

Se $i = j$

> $L_{jj} = \sqrt (A_{jj} -\sum_{k=1}^{j-1} L_{j,k}^2), j = 2, 3, ..., n$

Se $i \neq j$

> $L_{ij} = \sqrt (A_{ij} -\sum_{k=1}^{j-1} L_{i,k}*L_{j,k}) / L_{jj}, j = 2, 3, ..., n$

In [ ]:
# Recebe a matrix simetrica, positiva definida A = L @ (^t)
def choleskis_decomposition(a):  
  L = np.copy(A);
  n = len(a);

  for i in range(0, n):
    try:
      a[i, i] = np.sqrt(a[i, i]-np.dot(a[i, 0:i], a[i, 0:i]))
    except ValueError:
      error.err('Not a define posit]ive Matrix')  
    for k in range(i+1, n):
      a[k,i] = (a[k,i] - np.dot(a[k, 0:i], a[i, 0:i]))/a[i,i]
  for k in range(1, n):
    a[0:k, k] = 0.0
  return a

In [ ]:
def choleskis_solve(A, b):
  L = choleskis_decomposition(np.copy(A))
  U = transverse(np.copy(L))
  y = forwardSubstitution(L, b)
  return backSubstituion(U, y)


## Sistemas lineares diretos: Comparativo entre as funções e as bibliotecas do Python  

In [1]:
A = np.array([[2, 3], [5, 4]]);
b = np.array([8, 13]);
ans = np.array([1, 2]);

NameError: ignored

# **Métodos iterativos**

Para facilitar, usaremos majoritariamente matrizes não-singulares, $|A| != 0$ e portanto, haverá sempre ao menos uma solução.

## Gauss-Seidel: método iterativo.


In [ ]:
def gauss_seidel(x, n, tol=1.0e-9):
  w = 1.0
  p = 1
  k = 10
  for i in range(1,n):
    xOld = np.copy(x)
    x = interEqs(x, w)
    dx = np.sqrt(np.dot(x-xOld, x-xOld))
    if(dx < tol):
      return [x, i]
    if(i == k):
      dx1 = dx
    if(i == k + p):
      dx2 = dx
      w = 2.0/(1.0 + np.sqrt(1.0-(dx2/dx1)))

  print("No converge for ", tol)
  return  [-1]

## Gauss-Jacobi: método iterativo.




In [ ]:
def inter_eqs_jacobini(x, w = 1.0):
  # System Linear Solve with relatixion omega(w)
  x1 = np.zeros(len(x))
  for i in range(len(x)):
    x1[i] = (w*(b[i] - np.dot(a[i,:], x[:]) + np.dot(a[i, i], x[i]))/a[i,i])-((1-w)*x[i])
  return x1

In [ ]:
def gauss_jacobi(x, n, tol=1.0e-9):
  for i in range(1,n):
    xOld = np.copy(x)
    x  = inter_eqs_jacobini(x)
    dx = np.sqrt(np.dot(x-xOld, x-xOld))
    if(dx < tol):
      return [x, i]
  return [-1]

## Aplicações

## Calculo de densidades

![Provetas](https://www.researchgate.net/publication/321584117/figure/fig5/AS:569090473709574@1512693150597/Figura-5-Etapas-de-teste-de-proveta-e-regioes-formadas-durante-a-sedimentacao-FAUST.png)</br>imagem da analise de acumulo de sedimentos

A partir da relação $massa(m) = densidade(d) / volume(v)$, queremos descobrir a densidade $d_a$, $d_b$, $d_c$, $d_d$ dos respectivos liquidos A, B, C e D.</br> 
Porém apenas temos como estimar os volumes dos liquidos. Por exemplo, o liquido B na proveta 1, $v_{1b}$, na proveta 2, $v_{2b}$, do liquido A na proveta 2, $v_{2a}$, e assim adiante.</br>
Para cada proveta, também sabemos as massas contidas ali dentro, desconsiderando as massas das provetas, sendo estas respectivamente, $m_1$, $m_2$, $m_3$ e $m_4$. Com essas informações, temos o seguinte sistema:

> $v_{1a}*d_{a} + \cdots + v_{1d}*d_{d} = m_{1}$

> $v_{2a}*d_{a} + \cdots + v_{2d}*d_{d}= m_{2}$

> $v_{3a}*d_{a} + \cdots + v_{3d}*d_{d} = m_{3}$

> $a_{4a}*d_{a} + \cdots + v_{4d}*d_{d} = m_{4}$



# Referências

[1] Scientific Computing An Introductory Survey, capitulo 2, System of Linear Equations. 